## 테스트 데이터셋의 네 개 리뷰 중 어떤 것이 실제 인간에 의해 작성된 것인지 정확하게 예측
## 테스트 데이터셋의 'label' 필드를 복구

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModel

# - seed : 동일한 시드 값을 사용하면 항상 동일한 난수 시퀀스가 생성

In [3]:
def set_seed(seed=42):
    np.random.seed(seed)  # 이 부분이 pandas의 sample 함수에도 영향을 줍니다.
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

In [4]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_df.head()

,id,sentence1,sentence2,sentence3,sentence4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1


# - Kogpt2 (decoder only)
1. github : https://github.com/skt-ai/kogpt2?utm_medium=social&utm_source=velog&utm_campaign=everyone%20ai&utm_content=kogpt2
2. 설명1 : https://developers.kakao.com/docs/latest/ko/kogpt/common
3. 설명2 :https://velog.io/@yeop2/AI-%EB%AA%A8%EB%8D%B8-%ED%83%90%ED%97%98%EA%B8%B0-7-%ED%95%9C%EA%B8%80-%EB%B2%84%EC%A0%84%EC%9D%98-GPT-2-KoGPT2

In [5]:
# 현재 GPU가 사용 가능한지 확인합니다.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
# 사전 학습된 GPT-2 모델의 토크나이저, 토크나이저는 텍스트를 모델이 이해할 수 있는 형식으로 변환하는 역할
model = AutoModel.from_pretrained('skt/kogpt2-base-v2') # 동일한 사전 학습된 GPT-2 모델의 가중치를 로드
model.to(device) # 모델을 device로 이동

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

C:\Neuer Ordner\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\황의지\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


GPT2Model(
  (wte): Embedding(51200, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

# - Inference

train_df에서 행 하나를 랜덤하게 추출하여 example로 사용(one-shot)

실제 사람이 작성한 리뷰에 대해서는 '-> O'로 표시하고 그렇지 않은 리뷰에 대해서는 '-> X'로 표기

In [ ]:
model.eval() # 모델을 평가 모드로 설정, 학습 중에 사용되는 드롭아웃 및 배치 정규화 등의 기능이 비활성화되어 일관된 예측을 얻을 수 있음
preds = [] # 여기에 예측결과 저장됨

with torch.no_grad(): # 그레디언트 추적 비활성화, 모델을 평가할 때 불필요한 메모리 사용을 줄이고 속도를 향상
    
    # 각 '테스트' 케이스에 대해
    for idx in tqdm(range(len(test_df))):
        row = test_df.iloc[idx]
        best_score = float('-inf') # 음의 무한대로 초기화
        best_label = 0
        
        # 'train' 데이터에서 랜덤하게 문장을 가져옵니다.
        random_row = train_df.sample(1).iloc[0]
        random_answer = random_row['label']  #  random_row에서 랜덤하게 선택한 행의 '정답 레이블'
        random_labels = {}
        for i in range(1, 5):
            random_labels[f'sentence{i}'] = 'O' if i == random_answer else 'X' #  각각의 문장에 대한 레이블은 'O' 또는 'X' 중 하나로 설정, 각 문장이 정답인지 아닌지  
            
        # GPT-2에게 제공할 prompt를 작성합니다.
        example_sentence = f"""
        주어진 문장이 사람이 작성한 것이 맞으면 O, 아니면 X를 반환하세요. \

        # 예시

        문장1 : {random_row['sentence1']} -> {random_labels['sentence1']} \
        문장2 : {random_row['sentence2']} -> {random_labels['sentence2']} \
        문장3 : {random_row['sentence3']} -> {random_labels['sentence3']} \
        문장4 : {random_row['sentence4']} -> {random_labels['sentence4']} \

        # 문제
        문장 :
        """        

        # 각 문장(테스트)에 대한 확률값을 구하고, 가장 높은 확률값을 가진 문장을 선택합니다.
        for i in range(1, 5):
            prompt = example_sentence + " " + row[f"sentence{i}"]
            # 예를 들어, "주어진 문장이 사람이 작성한 것이 맞으면 O, 아니면 X를 반환하세요."와 "문장 : [현재 선택한 문장]"의 형식으로 구성
            inputs = tokenizer(prompt, return_tensors="pt")  # 토크나이징(형식변환)
            inputs = inputs.to(device) # 입력을 사용 중인 디바이스로 이동(GPU/CPU)
            with torch.no_grad(): # 메모리 사용 최적화
                outputs = model(**inputs)  # 모델을 사용하여 예측을 생성
                score = outputs[0][:, -1, :].max().item()  # 모델의 출력 중 가장 높은 값을 가져옴

            if score > best_score: # best_score 음의 무한대로 초기화 됐었음
                best_score = score
                best_label = i

        preds.append(best_label) 

 25%|███████████████████                                                          | 273/1100 [57:46<4:30:56, 19.66s/it]

`score = outputs[0][:, -1, :].max().item()` 이 코드는 GPT-2 모델의 출력에서 확률값을 추출하는 과정을 나타냅니다.

- `outputs[0]`: GPT-2 모델의 출력 중 첫 번째 부분을 선택합니다. 일반적으로 이 부분은 모델이 예측한 토큰에 대한 확률 분포를 나타냅니다.

- `[:, -1, :]`: 선택한 출력을 텐서의 슬라이싱(slice) 연산을 사용하여 조작합니다. 여기서 `[:, -1, :]`는 다음과 같이 해석됩니다:
    - `[:, ...]`: 모든 차원의 모든 요소를 선택합니다.
    - `-1`: 마지막 차원 (토큰의 차원)에서 마지막 요소를 선택합니다. 이것은 모델의 예측 중에서 마지막 토큰에 대한 정보를 가져옵니다.
    - `[:]`: 선택된 차원에서 모든 요소를 선택합니다.

- `.max().item()`: 선택된 확률 분포에서 가장 높은 확률값을 추출합니다. 이렇게 하면 모델이 예측한 다음 토큰 중에서 확률값이 가장 높은 토큰의 확률값을 얻게 됩니다.

따라서 `score` 변수에는 현재 선택한 문장이 다음 토큰을 예측할 때의 가장 높은 확률값이 저장됩니다. 이 확률값은 모델이 현재 문장을 얼마나 확신하고 있는지를 나타냅니다. 확률값이 더 높을수록 모델은 해당 문장을 예측에 사용할 가능성이 더 큽니다. 코드는 이러한 확률값을 기록하고, 가장 높은 확률값을 가진 문장을 선택하여 최종 예측을 수행하는 데 사용합니다.

# - Post-processing
본 대회에서는 각 케이스에 대한 예측값을 두 개 제출할 수 있기 때문에 나머지 하나를 채워줍니다.

베이스라인에서는 1,2,3,4 중에서 4를 예시로 사용했습니다.

In [ ]:
preds = [str(pred) + '4' for pred in preds]
preds[:5]

# - submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.head()

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)